# Homework 1

**Name:** Oscar Beltran Villegas

**e-mail:** oscar.beltran7944@alumnos.udg.mx

# MODULES

In [16]:
import math
import numpy as np
from scipy.stats import cauchy
import plotly.graph_objects as go

# CLASSES

In [17]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # Rotate vector
    def rotated(self, angle):
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        x = self.x * cos_theta - self.y * sin_theta
        y = self.x * sin_theta + self.y * cos_theta
        return Vec2d(x, y)

**Activity 1:** Correlated Random Walk - 1 Trajectory (**3 pts**)
- Select rotations from a **Cauchy** distribution (scipy).
- Implement usign **Vec2d** class.
- Keep trajectory in a **numpy** array
- Visualize trajectory using **plotly** - 2D projection

In [18]:
# Parameters
num_steps = 2000
speed = 3.0


trajectory = np.zeros((num_steps, 2))
# Init velocity vector
velocity = Vec2d(speed,0)

for i in range(1, num_steps):
    angle = cauchy.rvs()  # Rotation angle from Cauchy distribution
    velocity = velocity.rotated(angle)
    trajectory[i] = (trajectory[i - 1] + np.array([velocity.x, velocity.y]))

# Graficar la trayectoria
fig = go.Figure()
fig.add_trace(go.Scatter(x=trajectory[:, 0], y=trajectory[:, 1], mode='lines', name='Trajectory'))
fig.update_layout(title='Correlated Random Walk', xaxis_title='X', yaxis_title='Y')
fig.show()